In [1]:
!unzip final.zip

Archive:  final.zip
   creating: final/
   creating: final/raw-documents/
  inflating: final/raw-documents/PT_01.txt  
  inflating: final/raw-documents/PT_02.txt  
  inflating: final/raw-documents/PT_03.txt  
  inflating: final/raw-documents/PT_04.txt  
  inflating: final/raw-documents/PT_05.txt  
  inflating: final/raw-documents/PT_06.txt  
  inflating: final/raw-documents/PT_07.txt  
  inflating: final/raw-documents/PT_08.txt  
  inflating: final/raw-documents/PT_09.txt  
  inflating: final/raw-documents/PT_10.txt  
  inflating: final/raw-documents/PT_100.txt  
  inflating: final/raw-documents/PT_101.txt  
  inflating: final/raw-documents/PT_102.txt  
  inflating: final/raw-documents/PT_103.txt  
  inflating: final/raw-documents/PT_104.txt  
  inflating: final/raw-documents/PT_105.txt  
  inflating: final/raw-documents/PT_106.txt  
  inflating: final/raw-documents/PT_107.txt  
  inflating: final/raw-documents/PT_108.txt  
  inflating: final/raw-documents/PT_109.txt  
  inflating: fin

In [2]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
#importing libraries
import pandas as pd
import numpy as np
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import EvalPrediction


In [4]:
file_path = "/content/final/subtask-2-annotations.txt"
df = pd.read_csv(file_path, sep="\t", header=None, names=["Document_ID", "High_Level_Narratives", "Sub_Narratives"])

In [5]:
# Group by Document_ID and create lists of narratives
df = df.groupby("Document_ID").agg({
    "High_Level_Narratives": lambda x: list(set(";".join(x).split(";"))),
    "Sub_Narratives": lambda x: list(set(";".join(x).split(";")))
}).reset_index()

df.rename(columns={
    "High_Level_Narratives": "High_Level_Narratives_List",
    "Sub_Narratives": "Sub_Narratives_List"
}, inplace=True)

In [6]:
documents_folder = '/content/final/raw-documents'


def load_documents(folder_path):
    documents = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):  # Ensure only text files are read
            doc_id = filename  # Extract Document_ID from filename
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                documents[doc_id] = file.read().strip()
    return documents

# Load document texts into a dictionary
document_texts = load_documents(documents_folder)

# Map document texts to the DataFrame
df['Text'] = df['Document_ID'].map(document_texts)

# Check if all documents are successfully mapped
print(f"Number of missing documents: {df['Text'].isnull().sum()}")
df


Number of missing documents: 0


,Document_ID,High_Level_Narratives_List,Sub_Narratives_List,Text
0,PT_01.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Hidden plots by secret schemes of powerfu...,Ativistas climáticos ou idiotas úteis?\n\n“Pel...
1,PT_02.txt,"[URW: Discrediting the West, Diplomacy, URW: D...",[URW: Blaming the war on others rather than th...,Rússia: posição dos EUA sobre o ataque à Câmar...
2,PT_03.txt,"[URW: Discrediting Ukraine, URW: Praise of Rus...",[URW: Praise of Russia: Praise of Russian mili...,Pelo menos 20 mercenários polacos foram mortos...
3,PT_04.txt,[URW: Discrediting Ukraine],[URW: Discrediting Ukraine: Ukraine is a hub f...,Como é que notório terrorista do Daesh consegu...
4,PT_05.txt,[URW: Blaming the war on others rather than th...,[URW: Blaming the war on others rather than th...,Bundestag vota a favor do projeto de resolução...
...,...,...,...,...
395,PT_URW_414.txt,"[URW: Praise of Russia, URW: Amplifying war-re...",[URW: Blaming the war on others rather than th...,"Armas nucleares na Ucrânia, a responsabilidade..."
396,PT_URW_417.txt,"[URW: Discrediting Ukraine, URW: Blaming the w...",[URW: Blaming the war on others rather than th...,"""Tinha dito que ia chegar o momento em que Zel..."
397,PT_URW_418.txt,[CC: Criticism of institutions and authorities],[CC: Criticism of institutions and authorities...,OE2025: Climáximo critica orçamento que diz ga...
398,PT_URW_419.txt,"[URW: Russia is the Victim, URW: Praise of Rus...",[URW: Praise of Russia: Russia is a guarantor ...,Fórum em Ashkhabad. Putin reitera que nova ord...


In [7]:
# Convert the 'High_Level_Narratives_List' column from string to list
df['High_Level_Narratives_List'] = df['High_Level_Narratives_List'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Explode the dataframe to have one row per High Level Narrative
df = df.explode('High_Level_Narratives_List')

# Rename the column for clarity
df = df.rename(columns={"High_Level_Narratives_List": "High_Level_Narrative"})

df


,Document_ID,High_Level_Narrative,Sub_Narratives_List,Text
0,PT_01.txt,CC: Hidden plots by secret schemes of powerful...,[CC: Hidden plots by secret schemes of powerfu...,Ativistas climáticos ou idiotas úteis?\n\n“Pel...
1,PT_02.txt,"URW: Discrediting the West, Diplomacy",[URW: Blaming the war on others rather than th...,Rússia: posição dos EUA sobre o ataque à Câmar...
1,PT_02.txt,URW: Discrediting Ukraine,[URW: Blaming the war on others rather than th...,Rússia: posição dos EUA sobre o ataque à Câmar...
1,PT_02.txt,URW: Blaming the war on others rather than the...,[URW: Blaming the war on others rather than th...,Rússia: posição dos EUA sobre o ataque à Câmar...
2,PT_03.txt,URW: Discrediting Ukraine,[URW: Praise of Russia: Praise of Russian mili...,Pelo menos 20 mercenários polacos foram mortos...
...,...,...,...,...
398,PT_URW_419.txt,URW: Praise of Russia,[URW: Praise of Russia: Russia is a guarantor ...,Fórum em Ashkhabad. Putin reitera que nova ord...
399,PT_URW_420.txt,"URW: Discrediting the West, Diplomacy",[URW: Praise of Russia: Russia is a guarantor ...,Rússia defende legitimidade do lançamento de m...
399,PT_URW_420.txt,URW: Praise of Russia,[URW: Praise of Russia: Russia is a guarantor ...,Rússia defende legitimidade do lançamento de m...
399,PT_URW_420.txt,URW: Amplifying war-related fears,[URW: Praise of Russia: Russia is a guarantor ...,Rússia defende legitimidade do lançamento de m...


In [8]:
df = df[df["High_Level_Narrative"].str.startswith("URW")].reset_index(drop=True)
df.head()

,Document_ID,High_Level_Narrative,Sub_Narratives_List,Text
0,PT_02.txt,"URW: Discrediting the West, Diplomacy",[URW: Blaming the war on others rather than th...,Rússia: posição dos EUA sobre o ataque à Câmar...
1,PT_02.txt,URW: Discrediting Ukraine,[URW: Blaming the war on others rather than th...,Rússia: posição dos EUA sobre o ataque à Câmar...
2,PT_02.txt,URW: Blaming the war on others rather than the...,[URW: Blaming the war on others rather than th...,Rússia: posição dos EUA sobre o ataque à Câmar...
3,PT_03.txt,URW: Discrediting Ukraine,[URW: Praise of Russia: Praise of Russian mili...,Pelo menos 20 mercenários polacos foram mortos...
4,PT_03.txt,URW: Praise of Russia,[URW: Praise of Russia: Praise of Russian mili...,Pelo menos 20 mercenários polacos foram mortos...


In [9]:
print(f'We have {len(df)} CC High Level Narratives after spliting multiple URW High Level Narratives in multiple rows')

We have 605 CC High Level Narratives after spliting multiple URW High Level Narratives in multiple rows


In [10]:
#in here we will add high level narrative to text with a | seperating high level narrative and text
df["Text"] = df["High_Level_Narrative"] + " | " + df["Text"]

df.head()

,Document_ID,High_Level_Narrative,Sub_Narratives_List,Text
0,PT_02.txt,"URW: Discrediting the West, Diplomacy",[URW: Blaming the war on others rather than th...,"URW: Discrediting the West, Diplomacy | Rússia..."
1,PT_02.txt,URW: Discrediting Ukraine,[URW: Blaming the war on others rather than th...,URW: Discrediting Ukraine | Rússia: posição do...
2,PT_02.txt,URW: Blaming the war on others rather than the...,[URW: Blaming the war on others rather than th...,URW: Blaming the war on others rather than the...
3,PT_03.txt,URW: Discrediting Ukraine,[URW: Praise of Russia: Praise of Russian mili...,URW: Discrediting Ukraine | Pelo menos 20 merc...
4,PT_03.txt,URW: Praise of Russia,[URW: Praise of Russia: Praise of Russian mili...,URW: Praise of Russia | Pelo menos 20 mercenár...


In [11]:
import ast  # Safer alternative to eval()

def filter_sub_narratives(row):
    high_level = row["High_Level_Narrative"].strip()

    # Check if the value is already a list
    if isinstance(row["Sub_Narratives_List"], str):
        sub_narratives = ast.literal_eval(row["Sub_Narratives_List"])  # Safer than eval()
    else:
        sub_narratives = row["Sub_Narratives_List"]  # Already a list

    # Keep only sub-narratives that start with the high-level narrative
    filtered = [sub for sub in sub_narratives if sub.startswith(high_level)]
    return filtered

# Apply filtering function
df["Filtered_Sub_Narratives"] = df.apply(filter_sub_narratives, axis=1)
df['Filtered_Sub_Narratives']

,Filtered_Sub_Narratives
0,"[URW: Discrediting the West, Diplomacy: Other]"
1,[URW: Discrediting Ukraine: Other]
2,[URW: Blaming the war on others rather than th...
3,[URW: Discrediting Ukraine: Discrediting Ukrai...
4,[URW: Praise of Russia: Praise of Russian mili...
...,...
600,[URW: Praise of Russia: Russia is a guarantor ...
601,"[URW: Discrediting the West, Diplomacy: Other]"
602,[URW: Praise of Russia: Russia is a guarantor ...
603,[URW: Amplifying war-related fears: Other]


In [12]:
# Get all unique sub-narratives
sub_narratives = list(set(label for sublist in df["Filtered_Sub_Narratives"] for label in sublist))
len(sub_narratives)

41

In [13]:
def preprocess_multi_label(data, narratives):
    label_vectors = []
    for narratives_list in data['Filtered_Sub_Narratives']:
        vector = [1 if narrative in narratives_list else 0 for narrative in narratives]
        label_vectors.append(vector)
    data['Labels'] = label_vectors
    return data

# Encode sub-level narratives
df = preprocess_multi_label(df, sub_narratives)
df.head()


,Document_ID,High_Level_Narrative,Sub_Narratives_List,Text,Filtered_Sub_Narratives,Labels
0,PT_02.txt,"URW: Discrediting the West, Diplomacy",[URW: Blaming the war on others rather than th...,"URW: Discrediting the West, Diplomacy | Rússia...","[URW: Discrediting the West, Diplomacy: Other]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,PT_02.txt,URW: Discrediting Ukraine,[URW: Blaming the war on others rather than th...,URW: Discrediting Ukraine | Rússia: posição do...,[URW: Discrediting Ukraine: Other],"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,PT_02.txt,URW: Blaming the war on others rather than the...,[URW: Blaming the war on others rather than th...,URW: Blaming the war on others rather than the...,[URW: Blaming the war on others rather than th...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,PT_03.txt,URW: Discrediting Ukraine,[URW: Praise of Russia: Praise of Russian mili...,URW: Discrediting Ukraine | Pelo menos 20 merc...,[URW: Discrediting Ukraine: Discrediting Ukrai...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,PT_03.txt,URW: Praise of Russia,[URW: Praise of Russia: Praise of Russian mili...,URW: Praise of Russia | Pelo menos 20 mercenár...,[URW: Praise of Russia: Praise of Russian mili...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
! pip install scikit-multilearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 9.8 MB/s eta 0:00:00


In [15]:
from skmultilearn.model_selection import iterative_train_test_split
import numpy as np

def multilabel_train_test_split(X, Y, test_size=0.2):
    """
    Uses iterative stratification to split multi-label data while preserving label distributions.

    Parameters:
    - X: List or NumPy array (text data)
    - Y: NumPy array (multi-label format)
    - test_size: Fraction of test data

    Returns:
    - X_train, X_test, Y_train, Y_test
    """
    X = np.array(X).reshape(-1, 1)  # Reshape for compatibility
    Y = np.array(Y)

    X_train, Y_train, X_test, Y_test = iterative_train_test_split(X, Y, test_size)

    return X_train.ravel(), X_test.ravel(), Y_train, Y_test

# Convert labels to NumPy array
labels = np.array(df['Labels'].tolist(), dtype=np.float32)

# Convert indices to NumPy array for splitting
indices = np.arange(len(df))

# Perform iterative stratified split on indices
train_indices, train_labels, test_indices, test_labels = iterative_train_test_split(indices.reshape(-1, 1), labels, test_size=0.2)

# Flatten indices
train_indices = train_indices.ravel()
test_indices = test_indices.ravel()

# Extract corresponding texts using indices
train_texts = df.iloc[train_indices]["Text"].tolist()
test_texts = df.iloc[test_indices]["Text"].tolist()



In [16]:
import random

def handle_rare_classes(data, labels, min_samples=15, max_duplication=3):
    """
    Handles rare classes by undersampling majority classes and applying controlled oversampling
    with minor augmentations to prevent overfitting.

    - `min_samples`: Minimum number of instances per class after processing.
    - `max_duplication`: Maximum times a rare sample is duplicated.
    """
    label_sums = np.sum(labels, axis=0)
    rare_classes = np.where(label_sums < min_samples)[0]

    if len(rare_classes) == 0:
        print("No rare classes found. Skipping balancing.")
        return data, labels

    print(f"Rare classes found: {len(rare_classes)}. Applying undersampling & controlled oversampling...")

    # Convert to DataFrame for manipulation
    data_df = pd.DataFrame({"Text": data})

    # 1. **Undersample Majority Classes**
    label_counts = np.sum(labels, axis=0)
    majority_classes = np.where(label_counts > min_samples * 5)[0]  # Define threshold for majority

    if len(majority_classes) > 0:
        print(f"Undersampling {len(majority_classes)} majority classes...")
        drop_indices = []
        for maj_class in majority_classes:
            maj_indices = [i for i in range(len(labels)) if labels[i][maj_class] == 1]
            keep_size = min(len(maj_indices), min_samples * 5)  # Reduce but keep balanced
            drop_indices.extend(random.sample(maj_indices, len(maj_indices) - keep_size))

        # Drop excess samples
        data_df.drop(drop_indices, inplace=True)
        labels = np.delete(labels, drop_indices, axis=0)
        data_df.reset_index(drop=True, inplace=True)

    # 2. **Controlled Oversampling for Rare Classes**
    new_texts = []
    new_labels = []

    for rare_class in rare_classes:
        rare_indices = [i for i in range(len(labels)) if labels[i][rare_class] == 1]

        if len(rare_indices) == 0:
            continue  # Skip if no rare samples found

        num_to_add = min_samples - len(rare_indices)  # How many more samples we need
        num_to_add = min(num_to_add, max_duplication * len(rare_indices))  # Limit excessive duplication

        for _ in range(num_to_add):
            idx = random.choice(rare_indices)
            new_text = data_df.iloc[idx]["Text"]

            # **Minor Text Augmentation to Prevent Overfitting**
            if len(new_text) > 20:
                words = new_text.split()
                if len(words) > 5:
                    random.shuffle(words[:5])  # Shuffle only the first 5 words for variation
                new_text = " ".join(words)

            new_texts.append(new_text)
            new_labels.append(labels[idx])

    # Add new samples to dataset
    if len(new_texts) > 0:
        new_data_df = pd.DataFrame({"Text": new_texts})
        data_df = pd.concat([data_df, new_data_df], ignore_index=True)
        labels = np.vstack([labels, np.array(new_labels)])

    print(f"Final dataset size: {len(data_df)} samples")

    return data_df["Text"], labels.astype(np.float32)


In [17]:
import numpy as np

def print_label_distribution(labels):
    """Prints the count of positive instances for each class."""
    label_counts = np.sum(labels, axis=0)
    print("Label Counts:", label_counts)
    print("Min:", label_counts.min(), "Max:", label_counts.max())

# Check class distribution after applying handle_rare_classes
print("Before balancing:")
print_label_distribution(np.array(train_labels))

train_texts, train_labels = handle_rare_classes(train_texts, train_labels, min_samples=15, max_duplication=3)

print("After balancing:")
print_label_distribution(np.array(train_labels))


Before balancing:
Label Counts: [ 1. 12. 10.  3. 45. 52. 36.  4.  6.  1.  6. 17.  6.  1.  3. 22.  3. 27.
  9. 20. 14. 28. 70. 38. 23.  3.  6.  1. 24. 26. 50.  7. 12. 14. 10.  3.
  0. 16.  5. 18.  4.]
Min: 0.0 Max: 70.0
Rare classes found: 25. Applying undersampling & controlled oversampling...
Final dataset size: 639 samples
After balancing:
Label Counts: [ 4. 15. 19. 12. 51. 56. 42. 16. 15.  4. 15. 22. 20.  4. 15. 25. 12. 35.
 19. 20. 17. 30. 92. 42. 23. 12. 15.  4. 24. 30. 54. 16. 19. 15. 15. 13.
  0. 33. 15. 24. 15.]
Min: 0.0 Max: 92.0


In [18]:
from datasets import Dataset, DatasetDict

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({
    "text": train_texts,
    "labels": [np.array(lbl, dtype=np.float32).tolist() for lbl in train_labels]
})

test_dataset = Dataset.from_dict({
    "text": test_texts,
    "labels": [np.array(lbl, dtype=np.float32).tolist() for lbl in test_labels]
})

datasets = DatasetDict({"train": train_dataset, "test": test_dataset})

datasets = DatasetDict({"train": train_dataset, "test": test_dataset})


In [19]:
# from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

# Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# model = AutoModelForSequenceClassification.from_pretrained(
#     "roberta-base", num_labels=len(sub_narratives), problem_type="multi_label_classification"
# )

# model_name = "eduagarcia/RoBERTaCrawlPT-base"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Use XLM-RoBERTa which handles multiple languages including Portuguese
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=len(sub_narratives),
    problem_type="multi_label_classification"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize the dataset
tokenized_datasets = datasets.map(tokenize_function, batched=True)


Map:   0%|          | 0/639 [00:00<?, ? examples/s]

Map:   0%|          | 0/123 [00:00<?, ? examples/s]

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import EvalPrediction

def find_best_threshold(preds, labels):
    """Find the best threshold for F1-score optimization with finer granularity."""
    best_f1 = 0
    best_threshold = 0.1  # Adjusted default based on observed patterns
    # Focus on lower thresholds with smaller increments
    thresholds = np.arange(0.05, 0.3, 0.01)

    for threshold in thresholds:
        binarized_preds = (preds > threshold).astype(int)
        f1 = f1_score(labels, binarized_preds, average="weighted", zero_division=0)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold

def compute_metrics(p: EvalPrediction):
    """Compute metrics with enhanced threshold precision."""
    sigmoid_preds = torch.sigmoid(torch.tensor(p.predictions)).numpy()
    labels = p.label_ids

    best_threshold = find_best_threshold(sigmoid_preds, labels)
    preds = (sigmoid_preds > best_threshold).astype(int)

    precision = precision_score(labels, preds, average="weighted", zero_division=0)
    recall = recall_score(labels, preds, average="weighted", zero_division=0)
    f1 = f1_score(labels, preds, average="weighted", zero_division=0)

    return {
        "best_threshold": best_threshold,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [27]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,  # Increased training duration
    learning_rate=3e-5,  # Fine-tuned learning rate
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=1,
    seed=42,
    weight_decay=0.01,  # Added regularization
    fp16=True,  # Enabled mixed precision
    gradient_accumulation_steps=2,  # Simulates larger batch size
    warmup_ratio=0.1  # Added warmup
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)



<ipython-input-28-763004834412>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [29]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,Best Threshold,Precision,Recall,F1
1,No log,0.063706,0.060000,0.454523,0.981818,0.585062
2,0.130300,0.062800,0.120000,0.459029,0.909091,0.585072
3,0.130600,0.062972,0.110000,0.456249,0.957576,0.586741
4,0.127800,0.062376,0.130000,0.460387,0.927273,0.590337
5,0.123800,0.061390,0.130000,0.470216,0.909091,0.596732
6,0.123800,0.060738,0.110000,0.474235,0.903030,0.601235
7,0.116900,0.060192,0.130000,0.485468,0.878788,0.603526
8,0.112100,0.061212,0.170000,0.495480,0.866667,0.603720
9,0.107100,0.060146,0.120000,0.500471,0.909091,0.609332
10,0.104300,0.059372,0.140000,0.500888,0.848485,0.603437


TrainOutput(global_step=2400, training_loss=0.08706521411736806, metrics={'train_runtime': 2204.658, 'train_samples_per_second': 8.695, 'train_steps_per_second': 1.089, 'total_flos': 5045605108439040.0, 'train_loss': 0.08706521411736806, 'epoch': 30.0})

In [30]:
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# 🚀 1. Get model predictions on the test dataset
raw_predictions = trainer.predict(tokenized_datasets["test"])

# 🚀 2. Apply Sigmoid Activation (convert logits to probabilities)
sigmoid_preds = torch.sigmoid(torch.tensor(raw_predictions.predictions)).numpy()

# 🚀 3. Find the best threshold dynamically
def find_best_threshold(preds, labels):
    """Find the best threshold for F1-score optimization."""
    best_f1 = 0
    best_threshold = 0.5  # Default threshold
    thresholds = np.arange(0.1, 0.9, 0.05)  # Try thresholds from 0.1 to 0.9

    for threshold in thresholds:
        binarized_preds = (preds > threshold).astype(int)
        f1 = f1_score(labels, binarized_preds, average="weighted", zero_division=0)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold

# 🚀 4. Compute the best threshold
labels = np.array(test_labels)  # Convert test labels to NumPy array
best_threshold = find_best_threshold(sigmoid_preds, labels)

# 🚀 5. Convert probabilities to binary labels using the best threshold
binary_predictions = (sigmoid_preds > best_threshold).astype(int)

# 🚀 6. Compute evaluation metrics
precision = precision_score(labels, binary_predictions, average="weighted", zero_division=0)
recall = recall_score(labels, binary_predictions, average="weighted", zero_division=0)
f1 = f1_score(labels, binary_predictions, average="weighted", zero_division=0)

# 🚀 Reconstruct DataFrame using stored test indices
test_df = df.iloc[test_indices].copy()
test_df["Predicted Labels"] = binary_predictions.tolist()  # Add predictions
test_df["Actual Labels"] = test_labels.tolist()  # Add actual labels
test_df["Threshold Used"] = best_threshold  # Store the threshold for reference



# 🚀 Return Evaluation Metrics
evaluation_metrics = {
    "best_threshold": best_threshold,
    "precision": precision,
    "recall": recall,
    "f1": f1
}
evaluation_metrics



{'best_threshold': 0.20000000000000004,
 'precision': 0.57827351134489,
 'recall': 0.8,
 'f1': 0.6563600059304734}

In [31]:
# Function to map binary predictions to actual narrative labels
def convert_binary_predictions_to_labels(binary_predictions, label_names):
    """
    Converts binary predictions (0/1 array) into a list of predicted sub-narrative labels.

    Parameters:
    - binary_predictions: List of binary predictions (0/1)
    - label_names: List of all possible label names corresponding to each index

    Returns:
    - List of predicted labels for each row
    """
    return [[label_names[i] for i, pred in enumerate(pred_row) if pred == 1] for pred_row in binary_predictions]

# Extract unique sub-narratives (label names) from the dataset
unique_labels = df["Filtered_Sub_Narratives"].explode().unique().tolist()

# Convert predicted binary labels into human-readable sub-narrative labels
test_df["Predicted Sub_Narratives"] = convert_binary_predictions_to_labels(test_df["Predicted Labels"], unique_labels)

# Convert actual binary labels into human-readable sub-narrative labels
test_df["Actual Sub_Narratives"] = convert_binary_predictions_to_labels(test_df["Actual Labels"], unique_labels)

test_df=test_df[["Document_ID", "Predicted Sub_Narratives", "Actual Sub_Narratives"]]

test_df.to_csv("results_CC_sub.csv", index=False)


In [33]:
# 🚀 Reconstruct DataFrame using stored test indices
test_df = df.iloc[test_indices].copy()
test_df["Predicted Labels"] = binary_predictions.tolist()
test_df["Actual Labels"] = test_labels.tolist()
test_df["Threshold Used"] = best_threshold

# Convert labels AFTER creating the full test_df
test_df["Predicted Sub_Narratives"] = convert_binary_predictions_to_labels(test_df["Predicted Labels"], unique_labels)
test_df["Actual Sub_Narratives"] = convert_binary_predictions_to_labels(test_df["Actual Labels"], unique_labels)

# Save full results
test_df[["Document_ID", "Predicted Sub_Narratives", "Actual Sub_Narratives"]].to_csv("results_CC_sub.csv", index=False)

# Preview FIRST 15 rows to see index 14
test_df_preview = test_df[["Document_ID", "Predicted Sub_Narratives", "Actual Sub_Narratives"]].head(15)
test_df_preview.iloc[14]  # Use iloc for positional access

,30
Document_ID,PT_108.txt
Predicted Sub_Narratives,[URW: Discrediting Ukraine: Discrediting Ukrai...
Actual Sub_Narratives,[URW: Discrediting Ukraine: Discrediting Ukrai...


In [34]:
# Preview FIRST 15 rows to see index 14
test_df_preview = test_df[["Document_ID", "Predicted Sub_Narratives", "Actual Sub_Narratives"]].head(15).reset_index(drop=True)
test_df_preview.iloc[14]  # Now shows index 14 clearly

,14
Document_ID,PT_108.txt
Predicted Sub_Narratives,[URW: Discrediting Ukraine: Discrediting Ukrai...
Actual Sub_Narratives,[URW: Discrediting Ukraine: Discrediting Ukrai...


In [ ]:
!zip -r PT-expr-1.zip /content/results

  adding: content/results/ (stored 0%)
  adding: content/results/checkpoint-2240/ (stored 0%)
  adding: content/results/checkpoint-2240/trainer_state.json (deflated 80%)
  adding: content/results/checkpoint-2240/tokenizer_config.json (deflated 76%)
  adding: content/results/checkpoint-2240/scheduler.pt (deflated 55%)
  adding: content/results/checkpoint-2240/model.safetensors